In [1]:
# Importar as bibliotecas necessárias
import numpy as np # Para manipulação de arrays numéricos
from sklearn.model_selection import train_test_split # Para dividir os dados em treino e teste
from sklearn.preprocessing import StandardScaler     # Para escalar os dados
from sklearn.neighbors import KNeighborsClassifier   # O algoritmo KNN para classificação
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Métricas de avaliação

# --- 1. Geração de Dados de Exemplo ---
# Usamos dados sintéticos para demonstração.
# Em um cenário real, você carregaria seus dados de um arquivo (CSV, banco de dados, etc.).
# features: Duas características numéricas (ex: 'Idade', 'Renda')
# labels: Duas classes (0 ou 1, ex: 'Cliente Fiel', 'Cliente com Risco de Evasão')
np.random.seed(42) # Para reprodutibilidade
X = np.random.rand(100, 2) * 100 # 100 amostras, 2 características, valores entre 0 e 100
y = (X[:, 0] + X[:, 1] > 100).astype(int) # Uma regra simples para criar duas classes

print("--- Dados Iniciais ---")
print(f"Formato das features (X): {X.shape}") # Ex: (100, 2)
print(f"Formato dos rótulos (y): {y.shape}")   # Ex: (100,)
print(f"Primeiras 5 amostras de X:\n{X[:5]}")
print(f"Primeiros 5 rótulos de y: {y[:5]}\n")

# --- 2. Divisão dos Dados em Conjuntos de Treino e Teste ---
# Isso é crucial para avaliar a performance do modelo em dados não vistos.
# test_size=0.30 significa que 30% dos dados serão para teste, 70% para treino.
# random_state garante que a divisão seja a mesma cada vez que o código for executado.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print("--- Divisão dos Dados ---")
print(f"Formato dos dados de treino (X_train): {X_train.shape}")
print(f"Formato dos rótulos de treino (y_train): {y_train.shape}")
print(f"Formato dos dados de teste (X_test): {X_test.shape}")
print(f"Formato dos rótulos de teste (y_test): {y_test.shape}\n")

# --- 3. Escalonamento dos Dados (Padronização) ---
# O KNN calcula distâncias, então características com escalas diferentes
# podem distorcer os resultados. O StandardScaler transforma os dados para
# terem média 0 e desvio padrão 1.
# É importante AJUSTAR (fit) o scaler APENAS nos dados de TREINO para evitar vazamento de dados.
# E DEPOIS TRANSFORMAR (transform) ambos os conjuntos (treino e teste).
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("--- Dados Escalonados ---")
print(f"Primeiras 5 amostras de X_train_scaled:\n{X_train_scaled[:5]}\n")

# --- 4. Criação e Treinamento do Modelo KNN ---
# n_neighbors (K): Este é o número de vizinhos a serem considerados.
# É um hiperparâmetro crucial e pode ser ajustado.
# (Parâmetro mínimo, mas ajustável: n_neighbors=5 é um bom ponto de partida)
#
# Outros parâmetros comuns (para ajuste futuro):
# - weights: 'uniform' (todos os vizinhos têm o mesmo peso) ou 'distance' (vizinhos mais próximos pesam mais).
# - algorithm: 'auto', 'ball_tree', 'kd_tree', 'brute' (método para encontrar os vizinhos).
# - metric: 'euclidean', 'manhattan', 'minkowski' (tipo de distância a ser usada).
knn_model = KNeighborsClassifier(n_neighbors=5) # n_neighbors é o K

# 'Treinar' o modelo significa que ele memoriza os dados de treino escalados e seus rótulos.
knn_model.fit(X_train_scaled, y_train)

print("--- Modelo KNN Treinado ---\n")

# --- 5. Realização de Previsões no Conjunto de Teste ---
# Usamos o modelo treinado para prever os rótulos do conjunto de teste.
y_pred = knn_model.predict(X_test_scaled)

print("--- Previsões Realizadas ---")
print(f"Primeiras 10 previsões (y_pred): {y_pred[:10]}")
print(f"Primeiros 10 rótulos reais (y_test): {y_test[:10]}\n")

# --- 6. Avaliação do Desempenho do Modelo ---
# Comparamos as previsões com os rótulos reais para entender a performance.

# Acurácia: Proporção de previsões corretas.
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Modelo: {accuracy:.4f}\n")

# Relatório de Classificação: Fornece Precisão, Recall e F1-Score para cada classe.
# Precisão: Dos que foram previstos como positivos, quantos são realmente positivos.
# Recall: Dos que são realmente positivos, quantos foram corretamente identificados.
# F1-Score: Média harmônica entre Precisão e Recall.
print("--- Relatório de Classificação ---")
print(classification_report(y_test, y_pred))

# Matriz de Confusão: Tabela que mostra os acertos e erros do modelo.
# Linhas: Classes Reais; Colunas: Classes Previstas
# [[Verdadeiro Negativo, Falso Positivo],
#  [Falso Negativo, Verdadeiro Positivo]]
print("--- Matriz de Confusão ---")
print(confusion_matrix(y_test, y_pred))

# --- Exemplo de Previsão para um Novo Ponto de Dados ---
# Digamos que temos um novo cliente com 'Idade'=60 e 'Renda'=70.
novo_dado = np.array([[60, 70]])

# Importante: Escalar o novo dado usando o MESMO scaler que foi ajustado nos dados de treino.
novo_dado_escalado = scaler.transform(novo_dado)

# Fazer a previsão para o novo dado
nova_previsao = knn_model.predict(novo_dado_escalado)

# Obter as probabilidades para cada classe (opcional)
# Onde [0,1] significa [probabilidade da classe 0, probabilidade da classe 1]
nova_probabilidade = knn_model.predict_proba(novo_dado_escalado)

print("\n--- Previsão para um Novo Ponto de Dados ---")
print(f"Novo dado: {novo_dado[0]}")
print(f"Previsão para o novo dado (0 ou 1): {nova_previsao[0]}")
print(f"Probabilidade da Classe 0 (ex: 'Fiel'): {nova_probabilidade[0][0]:.4f}")
print(f"Probabilidade da Classe 1 (ex: 'Evasão'): {nova_probabilidade[0][1]:.4f}")

if nova_previsao[0] == 0:
    print("O novo dado é classificado como: Classe 0 (ex: Cliente Fiel)")
else:
    print("O novo dado é classificado como: Classe 1 (ex: Cliente com Risco de Evasão)")

--- Dados Iniciais ---
Formato das features (X): (100, 2)
Formato dos rótulos (y): (100,)
Primeiras 5 amostras de X:
[[37.45401188 95.07143064]
 [73.19939418 59.86584842]
 [15.60186404 15.59945203]
 [ 5.80836122 86.61761458]
 [60.11150117 70.80725778]]
Primeiros 5 rótulos de y: [1 1 0 0 1]

--- Divisão dos Dados ---
Formato dos dados de treino (X_train): (70, 2)
Formato dos rótulos de treino (y_train): (70,)
Formato dos dados de teste (X_test): (30, 2)
Formato dos rótulos de teste (y_test): (30,)

--- Dados Escalonados ---
Primeiras 5 amostras de X_train_scaled:
[[-0.55825816 -0.46668148]
 [ 0.93884271 -0.01941778]
 [ 0.64680666 -1.6943093 ]
 [-1.19500306 -1.06467323]
 [ 1.08346504  0.19610066]]

--- Modelo KNN Treinado ---

--- Previsões Realizadas ---
Primeiras 10 previsões (y_pred): [0 1 1 0 1 0 0 0 0 1]
Primeiros 10 rótulos reais (y_test): [0 1 1 0 1 0 0 1 0 1]

Acurácia do Modelo: 0.9667

--- Relatório de Classificação ---
              precision    recall  f1-score   support

   